
# Inferencia — Predicción de Consumo, Precio y Utilidad (sin gráficos)

Este cuaderno:
- Construye *features* (rezagos, medias móviles, dummies).
- Hace **partición temporal** (train/val/test) por año sin *leakage*.
- Entrena **baselines** y modelos (Ridge/Lasso/RandomForest; intenta LightGBM/XGBoost si están disponibles).
- Calcula métricas: **MAE, RMSE, MAPE, sMAPE, R²**.
- Guarda modelos `.joblib` y predicciones como CSV con columnas `country, type, year, target, y_true, y_pred, residual, model`.
- Resume resultados en texto.


In [ ]:
# %%
import warnings, json
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer 
import joblib

from utils.metrics import mae, rmse, r2, mape, smape, regression_report

DATA_CLEAN = "data/coffee_clean.csv"
MODELS_DIR = "models"
PRED_DIR = "predicciones"
Path(MODELS_DIR).mkdir(exist_ok=True)
Path(PRED_DIR).mkdir(exist_ok=True)

TARGETS = {
    "consumption": "consumption",
    "price": "price",
    "profit": "profit",
}

TEST_END = 2020
VAL_YEARS = 3
TEST_YEARS = 3


In [45]:
# %%
# Carga de datos
df = pd.read_csv(DATA_CLEAN)
df = df.sort_values(["country", "type", "year"]).reset_index(drop=True)

FEATURE_BASE = ["country", "type", "year"]
assert all(c in df.columns for c in FEATURE_BASE+["consumption","price","profit"]), "Faltan columnas requeridas"
df.head(3)


,country,type,consumption,year,price,revenue,profit,margin,market_share
0,Angola,Robusta/Arabica,1200000,1991,87.68636322,105223635.86425780,105130302.53092448,0.99911300,0.00102516
1,Angola,Robusta/Arabica,1800000,1992,87.68636322,157835453.79638672,157712120.46305338,0.99921860,0.00148309
2,Angola,Robusta/Arabica,2100000,1993,87.68636322,184141362.76245120,184003029.42911783,0.99924877,0.00167053


In [46]:
# %%
# Feature engineering SIN fuga (MAs solo con PASADO) + warm-up drop

LAGS = (1, 2, 3)
MAS  = (2, 3)  # ventanas de medias móviles

def add_lags_and_ma(g, cols, lags=LAGS, mas=MAS):
    """
    - Lags: shift(L) -> siempre pasado.
    - MAs: rolling(M).mean().shift(1) -> promedio hasta el año anterior (no incluye el actual).
    """
    g = g.copy()
    for col in cols:
        # Rezagos
        for L in lags:
            g[f"{col}_lag{L}"] = g[col].shift(L)
        # Medias móviles SÓLO con pasado
        for M in mas:
            g[f"{col}_ma{M}"] = g[col].rolling(M).mean().shift(1)
    return g

def build_features(df):
    parts = []
    for keys, gi in df.groupby(["country","type"], as_index=False):
        g2 = add_lags_and_ma(gi, cols=["consumption","price","profit"])
        parts.append(g2)
    X = (pd.concat(parts)
           .sort_values(["country","type","year"])
           .reset_index(drop=True))
    return X

X = build_features(df)

# Warm-up: eliminar filas iniciales por grupo donde aún existirán NaNs estructurales
# Por seguridad, usamos la ventana máxima de MA (M) y el lag máximo (L)
WARMUP = max(max(LAGS), max(MAS))
X = (X.groupby(["country","type"], group_keys=False)
       .apply(lambda g: g.iloc[WARMUP:])  # corta warm-up
       .reset_index(drop=True))

# Relleno conservador por si quedaron huecos (series con huecos originales)
X = (X.groupby(["country","type"], group_keys=False)
       .apply(lambda g: g.ffill().bfill())
       .reset_index(drop=True))

X.head(5)




,country,type,consumption,year,price,revenue,profit,margin,market_share,consumption_lag1,...,price_lag1,price_lag2,price_lag3,price_ma2,price_ma3,profit_lag1,profit_lag2,profit_lag3,profit_ma2,profit_ma3
0,Angola,Robusta/Arabica,1200000,1994,87.68636322,105223635.86425780,105130302.53092448,0.99911300,0.00094796,2100000.00000000,...,87.68636322,87.68636322,87.68636322,87.68636322,87.68636322,184003029.42911783,157712120.46305338,105130302.53092448,170857574.94608560,148948484.14103189
1,Angola,Robusta/Arabica,1500000,1995,87.68636322,131529544.83032228,131421211.49698894,0.99917636,0.00117040,1200000.00000000,...,87.68636322,87.68636322,87.68636322,87.68636322,87.68636322,105130302.53092448,184003029.42911783,157712120.46305338,144566665.98002118,148948484.14103189
2,Angola,Robusta/Arabica,600000,1996,87.68636322,52611817.93212891,52548484.59879557,0.99879621,0.00044243,1500000.00000000,...,87.68636322,87.68636322,87.68636322,87.68636322,87.68636322,131421211.49698894,105130302.53092448,184003029.42911783,118275757.01395673,140184847.81901041
3,Angola,Robusta/Arabica,1200000,1997,87.68636322,105223635.86425780,105130302.53092448,0.99911300,0.00085061,600000.00000000,...,87.68636322,87.68636322,87.68636322,87.68636322,87.68636322,52548484.59879557,131421211.49698894,105130302.53092448,91984848.04789227,96366666.20890301
4,Angola,Robusta/Arabica,2400000,1998,87.68636322,210447271.72851560,210293938.39518228,0.99927139,0.00164805,1200000.00000000,...,87.68636322,87.68636322,87.68636322,87.68636322,87.68636322,105130302.53092448,52548484.59879557,131421211.49698894,78839393.56486005,96366666.20890300


In [47]:
# %%
# Partición temporal sin leakage
min_year, max_year = int(X["year"].min()), int(X["year"].max())
test_start = max_year - TEST_YEARS + 1
val_end = test_start - 1
val_start = val_end - VAL_YEARS + 1
train_end = val_start - 1

splits = {
    "train": (min_year, train_end),
    "val": (val_start, val_end),
    "test": (test_start, max_year),
}
splits


{'train': (1994, 2014), 'val': (2015, 2017), 'test': (2018, 2020)}

In [48]:
# %%
# Baselines por grupo (país-tipo): último valor y promedio histórico
from utils.metrics import mae, rmse, r2, mape, smape

def baseline_last(g, target):
    return g[target].shift(1)

def baseline_mean(g, target):
    return g[target].shift(1).expanding().mean()

def compute_baselines(X, target):
    Xb = X.copy()
    Xb["y_true"] = Xb[target]
    Xb["y_last"] = (Xb.groupby(["country","type"], group_keys=False)
                      .apply(lambda g: baseline_last(g, target)))
    Xb["y_mean"] = (Xb.groupby(["country","type"], group_keys=False)
                      .apply(lambda g: baseline_mean(g, target)))
    return Xb

def evaluate_series(y_true, y_pred):
    # filtra pares válidos (sin NaN)
    m = (~pd.isna(y_true)) & (~pd.isna(y_pred))
    y_true = np.asarray(y_true[m], dtype=float)
    y_pred = np.asarray(y_pred[m], dtype=float)
    return {
        "MAE": mae(y_true, y_pred),
        "RMSE": rmse(y_true, y_pred),
        "MAPE": mape(y_true, y_pred),
        "sMAPE": smape(y_true, y_pred),
        "R2": r2(y_true, y_pred),
    }



In [49]:
def build_preprocessor(X):
    cat_features = ["country","type"]
    exclude = {"country","type","year","consumption","price","profit"}
    num_features = [c for c in X.columns if c not in exclude]

    num_pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler(with_mean=False)),  # opcional; RF no lo necesita, lineales sí
    ])

    pre = ColumnTransformer([
        ("cats", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_features),
        ("nums", num_pipe, num_features),
    ])
    return pre, num_features

def train_and_eval_model(X, target, model, model_name):
    pre, _ = build_preprocessor(X)
    y = X[target]
    feats = [c for c in X.columns if c not in ["consumption","price","profit"]]
    pipe = Pipeline([("pre", pre), ("model", model)])

    def mask_year(a, b): return (X["year"] >= a) & (X["year"] <= b)
    train_mask = mask_year(*splits["train"])
    val_mask   = mask_year(*splits["val"])
    test_mask  = mask_year(*splits["test"])

    # Entrenar en train+val sólo con filas donde y no es NaN
    trainval = (train_mask | val_mask) & y.notna()
    X_train, y_train = X.loc[trainval, feats], y.loc[trainval]

    pipe.fit(X_train, y_train)

    # Evaluar en test
    test_sel = test_mask & y.notna()
    X_test, y_test = X.loc[test_sel, feats], y.loc[test_sel]
    pred_test = pipe.predict(X_test)

    report = evaluate_series(y_test, pred_test)

    out_path = Path(MODELS_DIR) / f"{model_name}_{target}.joblib"
    joblib.dump(pipe, out_path)

    pred_df = (X.loc[test_sel, ["country","type","year"]].copy()
                 .assign(target=target, y_true=y_test.values, y_pred=pred_test))
    pred_df["residual"] = pred_df["y_true"] - pred_df["y_pred"]
    pred_df["model"] = model_name
    return report, out_path, pred_df



MODELS = {
    "ridge": Ridge(alpha=1.0, random_state=42),
    "lasso": Lasso(alpha=0.001, random_state=42, max_iter=10000),
    "rf": RandomForestRegressor(n_estimators=400, random_state=42, n_jobs=-1),
}

try:
    import lightgbm as lgb  # type: ignore
    MODELS["lgbm"] = lgb.LGBMRegressor(random_state=42, n_estimators=500)
except Exception:
    pass

try:
    import xgboost as xgb  # type: ignore
    MODELS["xgb"] = xgb.XGBRegressor(random_state=42, n_estimators=500, max_depth=6, subsample=0.8, colsample_bytree=0.8)
except Exception:
    pass


In [50]:
# %%
# Entrenamiento/evaluación por target
all_results = []
all_predictions = []

def eval_baselines_on_test(X, target):
    Xb = compute_baselines(X, target)
    a, b = splits["test"]
    mask = (Xb["year"] >= a) & (Xb["year"] <= b)
    base_last = evaluate_series(Xb.loc[mask,"y_true"], Xb.loc[mask,"y_last"])
    base_mean = evaluate_series(Xb.loc[mask,"y_true"], Xb.loc[mask,"y_mean"])
    return base_last, base_mean

baseline_summary = {}

for target in ["consumption","price","profit"]:
    base_last, base_mean = eval_baselines_on_test(X, target)
    baseline_summary[target] = {"baseline_last": base_last, "baseline_mean": base_mean}

from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor

results_rows = []
preds_parts = []

for target in ["consumption","price","profit"]:
    for name, model in MODELS.items():
        report, model_path, pred_df = train_and_eval_model(X, target, model, name)
        row = {"target": target, "model": name, **report}
        results_rows.append(row)
        preds_parts.append(pred_df)

results_df = pd.DataFrame(results_rows).sort_values(["target","RMSE"])
preds_df = pd.concat(preds_parts, ignore_index=True)

pred_file = Path(PRED_DIR) / "predicciones_test.csv"
preds_df.to_csv(pred_file, index=False)

results_df.head(10), pred_file, baseline_summary


(        target  model                MAE                RMSE        MAPE  \
 1  consumption  lasso   1240236.38290198    3640337.28110963 19.90339024   
 0  consumption  ridge   1161463.51704087    4021313.04891052 46.21771477   
 2  consumption     rf   4449790.95818182   22045747.82074357         NaN   
 3        price  ridge        12.20783213         13.92958505 11.45387484   
 4        price  lasso        12.27502778         14.03569552 11.50959567   
 5        price     rf        33.44606971         41.60884253 31.99505803   
 7       profit  lasso  21871034.93433449   61874438.72846415 39.59957615   
 6       profit  ridge  21828013.70829061   76711461.88990816 86.13954142   
 8       profit     rf 363062089.30081737 2518933228.41568136  1.06065944   
 
         sMAPE           R2  
 1 21.81387033   0.99960483  
 0 25.61407412   0.99951779  
 2  6.50561428   0.98550730  
 3 10.74520431  -8.36065029  
 4 10.81373227  -8.50380547  
 5 25.66781146 -82.52200882  
 7  9.20775197   0

In [52]:
# %%
# Resumen ejecutivo (mostrar métricas con más precisión)

def best_model_text_precise(results_df):
    lines = []
    for t, g in results_df.groupby("target"):
        g2 = g.sort_values("RMSE").iloc[0]
        lines.append(
            f"- **{t}**: mejor modelo = {g2['model']} "
            f"(RMSE={g2['RMSE']:.6g}, MAE={g2['MAE']:.6g}, sMAPE={g2['sMAPE']:.4f}%)."
        )
    return "\n".join(lines)

summary = "\n".join([
    "## Conclusiones",
    f"Rango temporal de entrenamiento/validación y prueba: {splits}.",
    "",
    "### Desempeño por objetivo",
    best_model_text_precise(results_df),
    "",
    "- Los baselines (`último valor` y `promedio histórico`) sirven de referencia; los modelos supervisados mejoran el error en la mayoría de casos.",
    "- **Consumo** suele beneficiarse de rezagos y MAs (definidas sólo con pasado).",
    "- **Precio** anualizado varía poco; comparar contra baselines es esencial.",
    "- **Utilidad** depende de supuestos de costos; revisar con finanzas `FIXED_COST` y `VAR_COST_PER_CUP`.",
    "",
    "Modelos y predicciones de test guardados en:",
    f"- Modelos: {MODELS_DIR}/",
    f"- Predicciones: {PRED_DIR}/predicciones_test.csv",
])

print(summary)


## Conclusiones
Rango temporal de entrenamiento/validación y prueba: {'train': (1994, 2014), 'val': (2015, 2017), 'test': (2018, 2020)}.

### Desempeño por objetivo
- **consumption**: mejor modelo = lasso (RMSE=3.64034e+06, MAE=1.24024e+06, sMAPE=21.8139%).
- **price**: mejor modelo = ridge (RMSE=13.9296, MAE=12.2078, sMAPE=10.7452%).
- **profit**: mejor modelo = lasso (RMSE=6.18744e+07, MAE=2.1871e+07, sMAPE=9.2078%).

- Los baselines (`último valor` y `promedio histórico`) sirven de referencia; los modelos supervisados mejoran el error en la mayoría de casos.
- **Consumo** suele beneficiarse de rezagos y MAs (definidas sólo con pasado).
- **Precio** anualizado varía poco; comparar contra baselines es esencial.
- **Utilidad** depende de supuestos de costos; revisar con finanzas `FIXED_COST` y `VAR_COST_PER_CUP`.

Modelos y predicciones de test guardados en:
- Modelos: models/
- Predicciones: predicciones/predicciones_test.csv


In [53]:
# %%
# Diagnóstico simple: correlación (Pearson) entre features y target por año (solo pasada vs. no pasada)
# Nota: esto es ilustrativo; no uses features del mismo año.

target = "consumption"
feats_same_year = [c for c in X.columns if c not in ["country","type","year","consumption","price","profit"]
                   and not c.endswith(tuple([f"_lag{L}" for L in LAGS]))
                   and "_ma" in c and True]  # MAs "sin shift" (si quedara alguna), aquí deberían ser 0

print("Features sospechosas (debería estar vacío si aplicaste shift a las MAs):", feats_same_year)


Features sospechosas (debería estar vacío si aplicaste shift a las MAs): ['consumption_ma2', 'consumption_ma3', 'price_ma2', 'price_ma3', 'profit_ma2', 'profit_ma3']
